In [ ]:
a = structure_DB.loc[0, 'init_structure']
a_copy = a.copy()
print(a_copy.cell.array,'\n',a_copy.cell.cellpar(),'\n',a_copy.positions)
print('jitter_ia')
jitter_ia(a_copy)
compute_distortion(a, a_copy, quiet = False)
print(a_copy.cell.array,'\n',a_copy.cell.cellpar(),'\n',a_copy.positions)
a_copy = a.copy()
print('jitter_ic')
jitter_ic(a_copy)
compute_distortion(a, a_copy, quiet = False)
print(a_copy.cell.array,'\n',a_copy.cell.cellpar(),'\n',a_copy.positions)

In [ ]:
for i in range(8):
    
    a = structure_DB.loc[i,'init_structure'].copy()
    master_copy = a.copy()
    print(master_copy.get_chemical_symbols())
    print(master_copy.cell.cellpar(),'\n',a_copy.positions)

    a_copy = a.copy()
    print('direct')
    relaxer_direct_2021_2_8_PES(a_copy)
    print(a_copy.cell.cellpar(),'\n',a_copy.positions)
    compute_distortion(master_copy, a_copy, quiet = False)
    
    print('jitter_ia')
    a_copy = a.copy()
    relaxer_jitter_a_01_2021_2_8_PES(a_copy)
    print(a_copy.cell.cellpar(),'\n',a_copy.positions)
    compute_distortion(master_copy, a_copy, quiet = False)
    
    print('jitter_ic')
    a_copy = a.copy()
    relaxer_jitter_c_005_2021_2_8_PES(a_copy)
    print(a_copy.cell.cellpar(),'\n',a_copy.positions)
    compute_distortion(master_copy, a_copy, quiet = False)
    break

In [ ]:
from ase.build import bulk
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.optimize import LBFGS


In [ ]:

rocksalt = bulk('NaCl', crystalstructure='rocksalt', a=6.0)

# Pseudopotentials from SSSP Efficiency v1.3.0
pseudopotentials = {'Na': 'na_pbe_v1.5.uspp.F.UPF', 'Cl': 'cl_pbe_v1.4.uspp.F.UPF'}

# Optionally create profile to override paths in ASE configuration:
profile = EspressoProfile(
    command='/path/to/pw.x', pseudo_dir='/path/to/pseudopotentials'
)

calc = Espresso(profile=profile, pseudopotentials=pseudopotentials)

rocksalt.calc = calc

rocksalt.get_potential_energy()  # This will run a single point calculation

opt = LBFGS(rocksalt)

opt.run(fmax=0.005)  # This will run a geometry optimization using ASE's LBFGS algorithm

# Print lattice constant...
print((8 * rocksalt.get_volume() / len(rocksalt)) ** (1.0 / 3.0))